In [ ]:
import os
from tqdm.auto import tqdm

import numpy as np
import torch
from torchvision import models
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import optim

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.style.use('seaborn-v0_8-colorblind')


import data_utils
from autoattack import AutoAttack
# (https://github.com/fra31/auto-attack/blob/master/autoattack/autoattack.py)


In [ ]:
##load model and defaalt image transform
model, preprocess0 = data_utils.get_target_model(target_name='resnet50', device='cuda', weights='default')
preprocess0


    

In [ ]:
##transforms
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

normalize = transforms.Normalize(mean=preprocess0.mean, std=preprocess0.std)

def denormalize(x):
    std = torch.tensor(preprocess0.std).view(3,1,1)
    mean = torch.tensor(preprocess0.mean).view(3,1,1)
    x = x * std + mean # de-normalize
    return x

to_pil = transforms.ToPILImage()


In [ ]:
##data, loader
dataset = data_utils.get_data('imagenet_val', preprocess)
loader = DataLoader(dataset, batch_size=50, shuffle=False, num_workers=8)

In [ ]:
# from inspect import getsource
# print(getsource(model._forward_impl))

# ## model forward pass
# def forward_pass(img):
#     img = normalize(img)
#     return model(img)

## model forward pass
# def forward_pass(img):
#     img = normalize(img)
    
#     x = img
#     x = model.conv1(x)
#     x = model.bn1(x)
#     x = model.relu(x)
#     x = model.maxpool(x)

#     x = model.layer1(x)
#     x = model.layer2(x)
#     x = model.layer3(x)
#     x = model.layer4(x)
        
#     return x

forward_pass = model

In [ ]:
## disable model training
for param in model.parameters():
    param.requires_grad_(False)
    

In [ ]:
a = torch.rand(4,3)
a.mean(1)

In [ ]:
def find_adversarial_examples(x, y, forward_pass, n_iter=10, vis=False, progress=True):
    x = x.clone().requires_grad_(True)
    optimizer = optim.SGD([x], lr=0.05)
    
    if progress:
        pbar = tqdm(range(n_iter))
    else:
        pbar = range(n_iter)
        
#     with torch.no_grad():
#         out = forward_pass(x)
#         target = out.roll(shifts=1, dims=1).detach_()
        
    for i in pbar:
        x.requires_grad_(True)
        out = forward_pass(x)
    
        ## minimize logit of the right class
        loss = out[torch.arange(x.shape[0]),y].sum()
        ## minimize logit of the neighbor of right classes
#         for i in range(-10,10):
#             loss += out[torch.arange(x.shape[0]),(y+i)%1000].sum()/20
        ## maximize logit of class + 500
        loss += -out[torch.arange(x.shape[0]),(y+500)%1000].sum()
        
        # mute neurons
#         loss = out.pow(2).sum()
        # scramble neurons
#         loss = (out-target).pow(2).sum()
    
        optimizer.zero_grad()
        loss.backward()
#         x.grad.data = x.grad.data.sign() # fast gradient sign
        optimizer.step()
        
        x.detach_()
#         x.clamp_(0,1)
        if progress:
            pbar.set_postfix({'loss':loss.item()})
        
        if vis:
            image_i = 0
            plt.subplot(131)
            plt.imshow(x[image_i].permute(1,2,0).detach().cpu().numpy())
#             plt.subplot(132)
#             plt.imshow(out[1,0].detach().cpu().numpy())
#             plt.colorbar()
#             plt.subplot(133)
#             plt.imshow(target[1,0].detach().cpu().numpy())
#             plt.colorbar()
            plt.subplot(132)
            plt.stem(out[image_i].detach().cpu().numpy())
            plt.show()
    return x

In [ ]:
## test for correctness

img, target = next(iter(loader))
img, target = img.cuda(), target.cuda()


a = find_adversarial_examples(img, target, forward_pass, vis=True)

print(model(img).argmax(1))
print(model(a).argmax(1))

plt.subplot(121)
plt.stem(model(img)[1].detach().cpu().numpy())
plt.subplot(122)
plt.stem(model(a)[1].detach().cpu().numpy())
plt.show()

---

In [ ]:
out_dir = '/home/lim38/dataset/imagenet-val-attack'
os.makedirs(out_dir, exist_ok=True)

In [ ]:
i = 0 # global image count
for k, [imgs, targets] in enumerate(tqdm(loader)):
    imgs, targets = imgs.cuda(), targets.cuda()
    advs = find_adversarial_examples(imgs, targets, forward_pass, vis=False, progress=False)
    for a in advs:
        pil = to_pil(a.detach().cpu())
        subdir, fn = dataset.imgs[i][0].split('/')[-2:]
        os.makedirs(f'{out_dir}/{subdir}', exist_ok=True)
        pil.save(f'{out_dir}/{subdir}/{fn}')
        i+=1

In [ ]:

# abs_diff = (a - img).abs()
# print('abs_diff.max', abs_diff.max())

# for i in range(8):
#     display(
#         'orignal',
#         to_pil(img[i].detach().cpu()), 
#         'orignal predict',
#         model(img[i:i+1]).argmax(1),

#         'adversarial',
#         to_pil(a[i].detach().cpu()),
#         'adversarial predict', 
#         model(a[i:i+1]).argmax(1),
#         '============='
#     )


In [ ]:
# out_dir = '/home/lim38/dataset/imagenet-val-attack'
# os.makedirs(out_dir, exist_ok=True)

# # subdirs = set([img_and_label[0].split('/')[-2] for img_and_label in dataset.imgs])
# # for subdir in subdirs:
# #     os.makedirs(f'{out_dir}/{subdir}', exist_ok=True)



In [ ]:
# adversary = AutoAttack(
#     forward_pass, 
#     norm='Linf', 
#     eps=5/255, #bound over image domain [0,1]
#     version='custom', 
#     attacks_to_run=['apgd-ce'],
#     verbose=True
# )

# loader = DataLoader(dataset, batch_size=64, shuffle=False)
# i = 0 # global image count
# for img, target in tqdm(loader):
    
#     img, target = img.cuda(), target.cuda()
#     adv = adversary.run_standard_evaluation(img, target)
    
#     for a in adv:
#         pil = to_pil(a.detach().cpu())
#         subdir, fn = dataset.imgs[i][0].split('/')[-2:]
#         os.makedirs(f'{out_dir}/{subdir}', exist_ok=True)
#         pil.save(f'{out_dir}/{subdir}/{fn}')
#         i+=1
